In [ ]:
# !pip install keras-tuner --upgrade

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 2.3 MB/s eta 0:00:00


In [ ]:
import keras_tuner
import keras_tuner as kt
import tensorflow.keras.backend as K
from tensorflow.keras.models import Model, clone_model
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard

In [ ]:
class CleanMemoryTuner(kt.RandomSearch):
    """Keras tuner wrapper to cleanup memory after each run."""
    def run_trial(self, trial, *args, **kwargs):
        K.clear_session()
        return super().run_trial(trial, *args, **kwargs)

In [ ]:
def model_builder_synthetic_se_cnn(hp, model_fun, input_dim):
    """
    Hypermodel function for CNN+SE tuning.
    """
    # Define search space for model architecture
    base_filters = hp.Choice("filter_scale", [0.5, 1.0, 1.5])
    filters = [int(f * base_filters) for f in [32, 64, 128, 128, 128]]
    dropout = hp.Float('dropout', 0.2, 0.6, step=0.2)
    se_ratio = hp.Choice('se_ratio', [8, 16])
    batch_normalize = hp.Boolean("batch_normalize")

    # Build the model using the passed-in function and input_dim
    model = model_fun(
        filters=filters,
        dropout_rate=dropout,
        se_ratio=se_ratio,
        batch_normalize=batch_normalize,
        input_dim=input_dim
    )

    # Define search space for optimizer type and learning rate
    optimizer_choice = hp.Choice("optimizer", ["adam", "rmsprop"])
    learning_rate = hp.Choice("learning_rate", [1e-4, 5e-4, 1e-3])

    if optimizer_choice == "adam":
        optimizer = Adam(learning_rate=learning_rate)
    else:
        optimizer = RMSprop(learning_rate=learning_rate)

    # Compile the model. Use 'sparse_categorical_crossentropy' for integer labels.
    model.compile(
        optimizer=optimizer,
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

In [5]:
def vgg_cbam_synthetic_builder(hp, model_fun, input_dim=701):
    """! Hypermodel function for CBAM model tuning.
    """
    base_filters = hp.Choice("filter_scale", [1.0, 1.5, 2.0])
    filters = [int(f * base_filters) for f in [64, 128, 256, 512, 512]]
    dropout = hp.Float('dropout', 0.0, 0.6, step=0.3)
    cbam_ratio = hp.Choice('cbam_ratio', [8, 16, 32])
    batch_normalize = hp.Boolean("batch_normalize")

    optimizer_choice = hp.Choice("optimizer", ["adam", "rmsprop"])
    learning_rate = hp.Choice("learning_rate", [1e-5, 5e-5, 1e-4, 5e-4])

    if optimizer_choice == "adam":
          optimizer = Adam(learning_rate=learning_rate)
    else:
          optimizer = RMSprop(learning_rate=learning_rate)

    model = model_fun(filters=filters,
                      dropout_rate=dropout,
                      cbam_ratio=cbam_ratio,
                      batch_normalize=batch_normalize,
                      optimizer=optimizer)

    model.compile(optimizer=optimizer,
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    return model